In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

In [2]:
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import train_test_split

In [3]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_val_predict

C:\Users\User\anaconda3\Lib\site-packages\dask\dataframe\_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [4]:
df = pd.read_csv('protine_nmbroto.csv')
df.head()

,0,0.108733546,0.181243666,-0.193199024,0.368659616,0.176360248,0.124401011,0.142956503,0.082627615,-0.208255894,0.003119164,0.317244611,-0.030887477,-0.268541309,-0.283266565,-0.196083409,0.067022171
0,0,-0.266394,0.233125,0.248388,-0.254646,0.096656,-0.403808,0.171464,-0.342012,0.249920,-0.045724,0.228849,0.004224,0.083314,-0.064141,0.044749,0.144704
1,0,0.097293,-0.097589,0.234659,-0.073352,0.149906,-0.034135,0.195248,0.040637,-0.030806,0.137653,0.225867,0.301799,-0.130358,0.231268,0.152561,0.253846
2,0,0.259802,0.031314,0.131695,0.137440,-0.313013,-0.207943,-0.232930,-0.166322,0.282500,0.048616,-0.025858,-0.344516,0.306204,-0.190972,-0.109005,-0.120020
3,0,0.078199,0.144526,-0.214015,0.045440,-0.169669,0.200328,-0.269732,0.204746,0.331284,-0.083756,-0.324572,0.101263,0.305438,0.091385,0.129002,0.020781
4,0,0.338603,0.242532,0.126678,0.226531,0.254125,0.043477,0.225944,0.047467,-0.344713,-0.154729,-0.140283,-0.555813,-0.303859,0.065183,0.210895,0.055767


In [5]:
y = df['0']
x = df.drop('0', axis=1) 

In [6]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.3)

In [7]:
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

models = [RandomForestClassifier(n_estimators = 200, max_depth = 10),
          XGBClassifier(n_estimators = 200,max_depth = 10, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 10,random_state = 50),
          GradientBoostingClassifier(n_estimators = 200, learning_rate = 0.5, random_state = 50),
          AdaBoostClassifier(n_estimators = 200, learning_rate = 0.1, random_state = 50)]
cv = KFold(n_splits=5, random_state=1, shuffle=True)
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, xtrain, ytrain, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(ytrain, pred)
  mcc = matthews_corrcoef(ytrain, pred)
  cm1 = confusion_matrix(ytrain, pred)
  kappa = cohen_kappa_score(ytrain, pred)
  f1 = f1_score(ytrain, pred)
  precision_score = precision_score(ytrain, pred)
  recall_score = recall_score(ytrain, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=10, n_estimat...  0.612839 -0.030328   
1  XGBClassifier(base_score=None, booster=None, c...  0.549649 -0.069703   
2      LGBMClassifier(max_depth=10, random_state=50)  0.549649 -0.051175   
3  GradientBoostingClassifier(learning_rate=0.5, ...  0.562688  0.013648   
4  AdaBoostClassifier(learning_rate=0.1, n_estima...  0.610832 -0.018027   

      Kappa  precision    recall        f1  sensitivity  specificity  
0 -0.018532   0.306452  0.052486  0.089623     0.932283     0.052486  
1 -0.065447   0.299539  0.179558  0.224525     0.760630     0.179558  
2 -0.049088   0.319502  0.212707  0.255390     0.741732     0.212707  
3  0.013476   0.373288  0.301105  0.333333     0.711811     0.301105  
4 -0.012109   0.333333  0.071823  0.118182     0.918110     0.071823  


In [8]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
x_ros, y_ros = ros.fit_resample(xtrain, ytrain)
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

models = [RandomForestClassifier(n_estimators = 200, max_depth = 10),
          XGBClassifier(n_estimators = 200,max_depth = 10, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 10,random_state = 50),
          GradientBoostingClassifier(n_estimators = 200, learning_rate = 0.5, random_state = 50),
          AdaBoostClassifier(n_estimators = 200, learning_rate = 0.1, random_state = 50)]
cv = KFold(n_splits=5, random_state=1, shuffle=True)
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, x_ros, y_ros, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(y_ros, pred)
  mcc = matthews_corrcoef(y_ros, pred)
  cm1 = confusion_matrix(y_ros, pred)
  kappa = cohen_kappa_score(y_ros, pred)
  f1 = f1_score(y_ros, pred)
  precision_score = precision_score(y_ros, pred)
  recall_score = recall_score(y_ros, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=10, n_estimat...  0.687402  0.376432   
1  XGBClassifier(base_score=None, booster=None, c...  0.700000  0.400310   
2      LGBMClassifier(max_depth=10, random_state=50)  0.682677  0.365849   
3  GradientBoostingClassifier(learning_rate=0.5, ...  0.677165  0.355923   
4  AdaBoostClassifier(learning_rate=0.1, n_estima...  0.595276  0.191720   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.374803   0.671470  0.733858  0.701279     0.640945     0.733858  
1  0.400000   0.692424  0.719685  0.705792     0.680315     0.719685  
2  0.365354   0.673653  0.708661  0.690714     0.656693     0.708661  
3  0.354331   0.661871  0.724409  0.691729     0.629921     0.724409  
4  0.190551   0.585816  0.650394  0.616418     0.540157     0.650394  
